###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es La Administradora de los Recursos del Sistema General de Seguridad - ADRES

In [1]:
import pandas as pd

###Descarga del dataset desde el endpoint

- archivo tq4m-hmg2.csv

Contiene la informacion la población activa del régimen contributivo que se encuentra almacenada en la Base de Datos Única de Afiliados, con corte al 6 de septiembre de 2024

In [2]:
contributive_health_system = pd.read_csv("https://www.datos.gov.co/resource/tq4m-hmg2.csv?$limit=3000000")
contributive_health_system.head(3)

,tps_gnr_nombre,tps_grp_etr_id,ent_id,ent_nombre,tps_rgm_nombre,tps_afl_nombre,tps_est_afl_nombre,tps_cnd_bnf_nombre,zns_nombre,dpr_nombre,mnc_nombre,tps_nvl_ssb_id,tps_grp_pbl_id,cantidad
0,Masculino,19 a 45,EPS048,ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SAL...,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CORDOBA,TIERRALTA,NaN,NaN,27
1,Femenino,50 a 55,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Rural,HUILA,GIGANTE,NaN,NaN,1
2,Masculino,19 a 45,EPS002,SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REG...,Contributivo,COTIZANTE,Protección Laboral C,NO APLICA,Rural,SUCRE,TOLU,NaN,NaN,2


- archivo d7a5-cnra.csv


Contiene la informacion la población activa del régimen subsidiado que se encuentra almacenada en la Base de Datos Única de Afiliados, con corte al 6 de septiembre de 2024

In [3]:
subsidized_health_system = pd.read_csv("https://www.datos.gov.co/resource/d7a5-cnra.csv?$limit=3000000")
subsidized_health_system.head(3)

,tps_gnr_nombre,tps_grp_etr_id,ent_id,ent_nombre,tps_rgm_nombre,tps_afl_nombre,tps_est_afl_nombre,tps_cnd_bnf_nombre,zns_nombre,dpr_nombre,mnc_nombre,tps_nvl_ssb_id,tps_grp_pbl_id,cantidad
0,Femenino,> 75,EPSS02,SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REG...,Subsidiado,CABEZA DE FAMILIA,Activo,NO APLICA,Urbana,META,VILLAVICENCIO,N,NIÑOS-NIÑAS-ADOLESCENTES Y JÓVENES EN PROCESO ...,3
1,Femenino,5 a 15,ESS024,COOSALUD EPS S.A.,Subsidiado,BENEFICIARIO,Activo,NO APLICA,Rural,ARAUCA,ARAUQUITA,1,RECIÉN NACIDO Y MENOR DE EDAD DE PADRES NO AFI...,1
2,Femenino,> 75,EPSS40,"ALIANZA MEDELLIN ANTIOQUIA EPS S.A.S. ""SAVIA S...",Subsidiado,CABEZA DE FAMILIA,Activo,NO APLICA,Rural,ANTIOQUIA,MONTEBELLO,2,POBLACIÓN CON SISBEN,21


## Analizando y transformando dataset para integrar a la base de datos
### Estudio de las columnas de cada dataset

In [4]:
# Extraer nombre columnas de los datasets
columns_cont = contributive_health_system.columns
columns_subs = subsidized_health_system.columns
print(columns_subs)
columns_cont

Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id', 'cantidad'],
      dtype='object')


Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id', 'cantidad'],
      dtype='object')

In [5]:
# Comparar columnas
common_columns = set(columns_cont).intersection(set(columns_subs))
print("Columnas comunes:", common_columns)

different_columns = set(columns_cont).symmetric_difference(set(columns_subs))
print("Columnas diferentes:", different_columns)


Columnas comunes: {'tps_grp_pbl_id', 'mnc_nombre', 'dpr_nombre', 'zns_nombre', 'tps_gnr_nombre', 'tps_rgm_nombre', 'tps_afl_nombre', 'tps_cnd_bnf_nombre', 'tps_est_afl_nombre', 'cantidad', 'ent_id', 'tps_grp_etr_id', 'ent_nombre', 'tps_nvl_ssb_id'}
Columnas diferentes: set()


In [6]:
# Descripción de columnas aplica para los dos datasets
column_descriptions = {
    "tps_gnr_nombre": ["Masculino", "Femenino"],
    "tps_grp_etr_id": {
        1: "< 1",
        2: "1 A 5",
        3: "6 A 15",
        4: "HOMBRES 16 A 19",
        5: "MUJER 16 A 19",
        6: "HOMBRES 20 A 45",
        7: "MUJER 20 A 45",
        8: "46 A 50",
        9: "51 A 55",
        10: "56 A 60",
        11: "61 A 65",
        12: "66 A 70",
        13: "71 A 75",
        14: "> 75"
    },
    "ent_id": "Código de 6 caracteres de la EPS (Ejemplo: EPS017, EPS002)",
    "ent_nombre": "Nombre de la EPS",
    "tps_rgm_nombre": ["Contributivo", "Subsidiado"],
    "tps_afl_nombre": ["Cotizante", "Beneficiario", "Adicional", "Cabeza de Familia"],
    "tps_est_afl_nombre": ["Activo", "Retirado", "Fallecido", "Suspendido", "Protección Laboral"],
    "tps_cnd_bnf_nombre": {"D": "Discapacidad", "E": "Escolaridad", "": "Vacío"},
    "zns_nombre": ["Rural", "Urbana"],
    "dpr_nombre": "Código de departamento asignado por el DANE",
    "mnc_nombre": "Código de municipio asignado por el DANE",
    "tps_nvl_ssb_id": {
        1: "NIVEL I",
        2: "NIVEL II",
        3: "NIVEL III",
        4: "NIVEL IV",
        "N": "NO APLICA POR MIGRACION"
    },
    "tps_grp_pbl_id": "Aplica solo para subsidiado",
    "cantidad": "Cantidad de registros que cumplen con las mismas variables"
}

In [7]:
# Verificar el numero de valores únicos que se guarda en cada columna contributivo
for col in columns_cont:
  print(f'{col}: {contributive_health_system[col].nunique()}')

tps_gnr_nombre: 2
tps_grp_etr_id: 12
ent_id: 32
ent_nombre: 32
tps_rgm_nombre: 1
tps_afl_nombre: 3
tps_est_afl_nombre: 2
tps_cnd_bnf_nombre: 3
zns_nombre: 2
dpr_nombre: 33
mnc_nombre: 1036
tps_nvl_ssb_id: 3
tps_grp_pbl_id: 19
cantidad: 3155


In [8]:
# Verificar el numero de valores únicos que se guarda en cada columna subsidiad
for col in columns_subs:
  print(f'{col}: {subsidized_health_system[col].nunique()}')

tps_gnr_nombre: 2
tps_grp_etr_id: 12
ent_id: 30
ent_nombre: 30
tps_rgm_nombre: 1
tps_afl_nombre: 3
tps_est_afl_nombre: 1
tps_cnd_bnf_nombre: 3
zns_nombre: 2
dpr_nombre: 33
mnc_nombre: 1037
tps_nvl_ssb_id: 10
tps_grp_pbl_id: 25
cantidad: 2702


In [9]:
# Obtener las columnas que no son numéricas
non_numeric_columns_cont = contributive_health_system.select_dtypes(exclude=['int64', 'float64']).columns
print(non_numeric_columns_cont)

non_numeric_columns_subs = subsidized_health_system.select_dtypes(exclude=['int64', 'float64']).columns
print(non_numeric_columns_subs)

Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id'],
      dtype='object')
Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id'],
      dtype='object')


In [10]:
print(contributive_health_system['tps_gnr_nombre'].unique())
print(subsidized_health_system['tps_gnr_nombre'].unique())

['Masculino' 'Femenino']
['Femenino' 'Masculino']


In [11]:
print(contributive_health_system['tps_grp_etr_id'].unique())
print(subsidized_health_system['tps_grp_etr_id'].unique())

['19 a 45' '50 a 55' '65 a 70' '55 a 60' '1 a 5' '60 a 65' '> 75'
 '45 a 50' '5 a 15' '< 1' '70 a 75' '15 a 19']
['> 75' '5 a 15' '45 a 50' '50 a 55' '< 1' '1 a 5' '55 a 60' '65 a 70'
 '15 a 19' '70 a 75' '60 a 65' '19 a 45']


In [12]:
print(contributive_health_system['ent_id'].unique())
print(subsidized_health_system['ent_id'].unique())

['EPS048' 'EPS037' 'EPS002' 'ESSC24' 'EPS042' 'EPS005' 'CCFC33' 'EPS017'
 'CCFC55' 'ESSC18' 'EPS008' 'EPS041' 'EPSIC5' 'ESSC07' 'EPS010' 'EPS040'
 'ESSC62' 'EPSC34' 'EPSIC4' 'CCFC50' 'EPS018' 'CCFC20' 'EPSC25' 'EPSIC6'
 'EAS027' 'EPSIC3' 'EAS016' 'EPSIC1' 'EPS001' 'EPS012' 'EPS046' 'EPS047']
['EPSS02' 'ESS024' 'EPSS40' 'EPSS08' 'EPSS41' 'EPSS37' 'CCF050' 'ESS207'
 'EPSS05' 'ESS062' 'EPSS17' 'CCF033' 'EPSS12' 'EPSI03' 'EPSS10' 'ESS118'
 'CCF102' 'EPS025' 'CCF055' 'EPSS42' 'EPSI05' 'EPSS34' 'EPSI06' 'EPSI04'
 'EPSS18' 'EPSS48' 'EPSI01' 'EPSS01' 'EPSS46' 'EPSS47']


In [13]:
print(contributive_health_system['ent_nombre'].unique())
print(subsidized_health_system['ent_nombre'].unique())

['ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS'
 'NUEVA EPS S.A.'
 'SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.'
 'COOSALUD EPS S.A. -CM' 'COOSALUD EPS S.A.'
 'ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.'
 'EPS FAMILIAR DE COLOMBIA S.A.S. -CM' 'EPS FAMISANAR S.A.S.'
 'CAJACOPI EPS S.A.S -CM' 'EMSSANAR S.A.S. -CM'
 'CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR' 'NUEVA EPS S.A. -CM'
 'ENTIDAD PROMOTORA DE SALUD MALLAMAS EPSI -CM'
 'ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD ENTIDAD PROMOTORA DE SALUD - MUTUAL SER EPS -CM'
 'EPS SURAMERICANA S.A.'
 'ALIANZA MEDELLIN ANTIOQUIA EPS S.A.S. "SAVIA SALUD EPS" -CM'
 'ASMET SALUD EPS S.A.S. -CM'
 'CAPITAL SALUD ENTIDAD PROMOTORA DE SALUD DEL RÉGIMEN SUBSIDIADO SAS "CAPITAL SALUD EPS-S S.A.S." -CM'
 'EMPRESA PROMOTORA DE SALUD INDÍGENA ANAS WAYUU EPSI -CM'
 'CAJA DE COMPENSACIÓN FAMILIAR DEL ORIENTE COLOMBIANO "COMFAORIENTE" -CM'
 'ENTIDAD PROMOTORA D

In [14]:
print(contributive_health_system['tps_rgm_nombre'].unique())
print(subsidized_health_system['tps_rgm_nombre'].unique())

['Contributivo']
['Subsidiado']


In [15]:
print(contributive_health_system['tps_afl_nombre'].unique())
print(subsidized_health_system['tps_afl_nombre'].unique())

['COTIZANTE' 'BENEFICIARIO' 'ADICIONAL']
['CABEZA DE FAMILIA' 'BENEFICIARIO' 'OTRO MIEMBRO DEL NUCLEO FAMILIAR']


In [16]:
print(contributive_health_system['tps_est_afl_nombre'].unique())
print(subsidized_health_system['tps_est_afl_nombre'].unique())

['Activo' 'Protección Laboral C']
['Activo']


In [17]:
print(contributive_health_system['tps_cnd_bnf_nombre'].unique())
print(subsidized_health_system['tps_cnd_bnf_nombre'].unique())

['NO APLICA' 'ESTUDIANTE' 'DISCAPACIDAD']
['NO APLICA' 'ESTUDIANTE' 'DISCAPACIDAD']


In [18]:
print(contributive_health_system['zns_nombre'].unique())
print(subsidized_health_system['zns_nombre'].unique())

['Urbana' 'Rural']
['Urbana' 'Rural']


In [19]:
print(contributive_health_system['dpr_nombre'].unique())
print(subsidized_health_system['dpr_nombre'].unique())

['CORDOBA' 'HUILA' 'SUCRE' 'SANTANDER' 'CHOCO' 'BOGOTA D.C.' 'ANTIOQUIA'
 'BOLIVAR' 'TOLIMA' 'QUINDIO' 'CESAR' 'PUTUMAYO' 'BOYACA' 'CAUCA'
 'RISARALDA' 'MAGDALENA' 'CUNDINAMARCA' 'NARINO' 'META' 'VAUPES' 'ARAUCA'
 'ATLANTICO' 'LA GUAJIRA' 'NORTE DE SANTANDER' 'VALLE' 'CASANARE'
 'CAQUETA' 'CALDAS' 'VICHADA' 'AMAZONAS' 'GUAVIARE' 'GUAINIA' 'SAN ANDRES']
['META' 'ARAUCA' 'ANTIOQUIA' 'VALLE' 'TOLIMA' 'CHOCO' 'CUNDINAMARCA'
 'NORTE DE SANTANDER' 'BOLIVAR' 'CAUCA' 'CALDAS' 'RISARALDA' 'HUILA'
 'MAGDALENA' 'SUCRE' 'LA GUAJIRA' 'NARINO' 'SANTANDER' 'CORDOBA' 'BOYACA'
 'CESAR' 'PUTUMAYO' 'QUINDIO' 'CASANARE' 'CAQUETA' 'ATLANTICO' 'GUAINIA'
 'BOGOTA D.C.' 'VICHADA' 'GUAVIARE' 'AMAZONAS' 'VAUPES' 'SAN ANDRES']


In [20]:
print(contributive_health_system['mnc_nombre'].unique())
print(subsidized_health_system['mnc_nombre'].unique())

['TIERRALTA' 'GIGANTE' 'TOLU' ... 'LA GUADALUPE' 'PAPUNAHUA'
 'DPTO. AMAZONAS']
['VILLAVICENCIO' 'ARAUQUITA' 'MONTEBELLO' ... 'YAVARATE' 'PAPUNAHUA'
 'DPTO. AMAZONAS']


In [21]:
print(contributive_health_system['tps_nvl_ssb_id'].unique())
print(subsidized_health_system['tps_nvl_ssb_id'].unique())

[nan 'N' '1' '2']
['N' '1' '2' nan 'C' '0' 'A' 'O' 'B' '3' '4']


In [22]:
print(contributive_health_system['tps_grp_pbl_id'].unique())
print(subsidized_health_system['tps_grp_pbl_id'].unique())

[nan 'VÍCTIMAS DEL CONFLICTO ARMADO INTERNO' 'POBLACIÓN CON SISBEN'
 'COMUNIDADES INDÍGENAS'
 'LOS VOLUNTARIOS ACREDITADOS Y ACTIVOS DE LA DEFENSA CIVIL COLOMBIANA CRUZ ROJA COLOMBIANA Y CUERPOS DE BOMBEROS'
 'HABITANTE DE LA CALLE' 'ADULTO MAYOR EN CENTROS DE PROTECCIÓN'
 'NIÑOS-NIÑAS-ADOLESCENTES Y JÓVENES EN PROCESO ADMINISTRATIVO PARA EL RESTABLECIMIENTO DE SUS DERECHOS'
 'MIGRANTES COLOMBIANOS REPATRIADOS QUE HAN RETORNADO VOLUNTARIAMENTE AL PAÍS O HAN SIDO DEPORTADOS O EXPULSADOS DE TERRITORIO EXTRANJERO.'
 'PERSONAS EN PRISIÓN DOMICILIARIA A CARGO DEL INPEC.'
 'POBLACIÓN RECLUSA A CARGO DE LA ENTIDAD TERRITORIAL' 'RROM (GITANO)'
 'POBLACIÓN INFANTIL VULNERABLE BAJO PROTECCIÓN DE INSTITUCIONES DIFERENTES AL ICBF'
 'PROGRAMA EN PROTECCIÓN A TESTIGOS'
 'RECIÉN NACIDO Y MENOR DE EDAD DE PADRES NO AFILIADOS'
 'POBLACIÓN PERTENECIENTE AL SISTEMA DE RESPONSABILIDAD PENAL PARA ADOLESCENTES A CARGO DEL ICBF'
 'PERSONAS QUE DEJEN DE SER MADRES COMUNITARIAS.'
 'MENORES DESVINCULADOS DEL CO

In [23]:
# Integrar los datasets verticalmente
health_insurance_affiliated = pd.concat([contributive_health_system, subsidized_health_system], ignore_index=True)

# Verificar que se combinaron correctamente
print(health_insurance_affiliated.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264095 entries, 0 to 1264094
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   tps_gnr_nombre      1264095 non-null  object
 1   tps_grp_etr_id      1264095 non-null  object
 2   ent_id              1264095 non-null  object
 3   ent_nombre          1264095 non-null  object
 4   tps_rgm_nombre      1264095 non-null  object
 5   tps_afl_nombre      1264095 non-null  object
 6   tps_est_afl_nombre  1264095 non-null  object
 7   tps_cnd_bnf_nombre  1264095 non-null  object
 8   zns_nombre          1264095 non-null  object
 9   dpr_nombre          1264095 non-null  object
 10  mnc_nombre          1264095 non-null  object
 11  tps_nvl_ssb_id      1005410 non-null  object
 12  tps_grp_pbl_id      994303 non-null   object
 13  cantidad            1264095 non-null  int64 
dtypes: int64(1), object(13)
memory usage: 135.0+ MB
None


In [24]:
# Eliminar columnas que se considera innecesarias para el proyecto
delete_columns = [ ]# Eliminando las columnas especificadas
health_insurance_affiliated = health_insurance_affiliated.drop(delete_columns, axis=1)

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [25]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [26]:
# columns categoricas
categorical_col = health_insurance_affiliated.select_dtypes(include=['object']).columns

In [27]:
# Aplicar la función a todas las columnas categóricas
health_insurance_affiliated[categorical_col] = health_insurance_affiliated[categorical_col].astype(str)
for col in categorical_col:
    health_insurance_affiliated[col] = health_insurance_affiliated[col].apply(remove_accents_and_special_chars)

In [28]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in categorical_col:
  l = health_insurance_affiliated[col].nunique()
  if l < 50:
    print(f'{col}: {health_insurance_affiliated[col].unique()}')

tps_gnr_nombre: ['MASCULINO' 'FEMENINO']
tps_grp_etr_id: ['19 A 45' '50 A 55' '65 A 70' '55 A 60' '1 A 5' '60 A 65' '> 75'
 '45 A 50' '5 A 15' '< 1' '70 A 75' '15 A 19']
tps_rgm_nombre: ['CONTRIBUTIVO' 'SUBSIDIADO']
tps_afl_nombre: ['COTIZANTE' 'BENEFICIARIO' 'ADICIONAL' 'CABEZA DE FAMILIA'
 'OTRO MIEMBRO DEL NUCLEO FAMILIAR']
tps_est_afl_nombre: ['ACTIVO' 'PROTECCION LABORAL C']
tps_cnd_bnf_nombre: ['NO APLICA' 'ESTUDIANTE' 'DISCAPACIDAD']
zns_nombre: ['URBANA' 'RURAL']
dpr_nombre: ['CORDOBA' 'HUILA' 'SUCRE' 'SANTANDER' 'CHOCO' 'BOGOTA D.C.' 'ANTIOQUIA'
 'BOLIVAR' 'TOLIMA' 'QUINDIO' 'CESAR' 'PUTUMAYO' 'BOYACA' 'CAUCA'
 'RISARALDA' 'MAGDALENA' 'CUNDINAMARCA' 'NARINO' 'META' 'VAUPES' 'ARAUCA'
 'ATLANTICO' 'LA GUAJIRA' 'NORTE DE SANTANDER' 'VALLE' 'CASANARE'
 'CAQUETA' 'CALDAS' 'VICHADA' 'AMAZONAS' 'GUAVIARE' 'GUAINIA' 'SAN ANDRES']
tps_nvl_ssb_id: ['NAN' 'N' '1' '2' 'C' '0' 'A' 'O' 'B' '3' '4']
tps_grp_pbl_id: ['NAN' 'VICTIMAS DEL CONFLICTO ARMADO INTERNO' 'POBLACION CON SISBEN'
 'COMUN

In [29]:
# Función para convertir rangos de edad en categorías
def categorize_age_group(age_range):
    if '<' in age_range or '1 A 5' in age_range or '5 A 15' in age_range:
        return 'NIÑEZ'
    elif '15 A 19' in age_range or '19 A 45' in age_range:
        return 'JUVENTUD'
    elif '45 A 50' in age_range or '50 A 55' in age_range or '55 A 60' in age_range or '60 A 65' in age_range:
        return 'ADULTEZ'
    elif '65 A 70' in age_range or '70 A 75' in age_range or '> 75' in age_range:
        return 'VEJEZ'
    else:
        return 'SIN CLASIFICAR'

# Aplicar la función para crear una nueva columna categórica
health_insurance_affiliated['tps_grp_etr_id'] = health_insurance_affiliated['tps_grp_etr_id'].apply(categorize_age_group)

health_insurance_affiliated['tps_grp_etr_id'].unique()

array(['JUVENTUD', 'ADULTEZ', 'VEJEZ', 'NIÑEZ'], dtype=object)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios no existen, asi que tenemos que originarlos utilizando las columnas que tienen los nombres de los departamentos y municipios.


Para hacer esta operacion, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para aplicar la coincidencia difusa para generar los codigos en una nueva columna

In [30]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [31]:
# Aplicar funcion remove_accents_and_special_chars a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [32]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [33]:
health_insurance_affiliated.columns

Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id', 'cantidad'],
      dtype='object')

  - Comparar nombres de departamentos en los dataframes

In [34]:
# Comparar listas de departamento
compare_lists(health_insurance_affiliated['dpr_nombre'], dept_mpios_codes['dept_name'],
              "Departamentos en health_insurance_affiliated", "Departamentos dept_mpios_codes")

Departamentos en health_insurance_affiliated que no están en Departamentos dept_mpios_codes:
{'SAN ANDRES', 'VALLE'}


  - Solucionar discrepancias en nombres de departamento

In [35]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA'}

# Reemplazar los nombres incorrectos  el dataframe health_insurance_affiliated
health_insurance_affiliated['dpr_nombre'] = health_insurance_affiliated['dpr_nombre'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [36]:
# Comparar listas de departamentos nuevamente
compare_lists(health_insurance_affiliated['dpr_nombre'], dept_mpios_codes['dept_name'],
              "Departamentos en health_insurance_affiliated", "Departamentos dept_mpios_codes")

Departamentos en health_insurance_affiliated que no están en Departamentos dept_mpios_codes:
set()


In [37]:
# Comparar listas de departamentos nuevamente
compare_lists(health_insurance_affiliated['mnc_nombre'], dept_mpios_codes['mupio_name'],
              "Departamentos en health_insurance_affiliated", "Departamentos dept_mpios_codes")

Departamentos en health_insurance_affiliated que no están en Departamentos dept_mpios_codes:
{'DPTO. AMAZONAS', 'CANTON DE SAN PABLO', 'S.ANTONIO TEQUENDAMA', 'PTO SANTANDER', 'PATIA (EL BORDO)', 'SAN JUAN DE RIO SECO', 'MOMPOS', 'BUGA', 'SAN SEBASTIAN BUENAVIST', 'SAN VICENTE', 'PUERTO  LEGUIZAMO', 'SAN JOSE DE CALDAS', 'PIENDAMO', 'MARIQUITA', 'SAN JOSE DE FRAGUA', 'EL TABLON', 'UBATE', 'VISTA HERMOSA', 'TOLU', 'CERRO SAN ANTONIO', 'FRANCIS PIZARRO', 'CARTAGENA', 'ITSMINA', 'CODAZZI', 'DOS QUEBRADAS', 'EL PIJI¥O DEL CARMEN', 'SINCE', 'LA URIBE', 'NARI¥O', 'SANTIAGO DE CALI', 'TOLUVIEJO', 'VILLA HERMOSA', 'DPTO. GUAINIA', 'GUICAN', 'DON MATIAS', 'POLO NUEVO', 'MORICHAL NUEVO', 'BARRANCO MINAS', 'DPTO. VAUPES', 'TUMACO', 'TUTASA', 'SOTARA', 'GUAYABAL', 'LEYVA', 'LA PAZ  ROBLES', 'MIRITINPARANA', 'CARMEN DE VIBORAL', 'CUASPUD', 'PURISIMA', 'SABANA DE SAN ANGEL', 'ANTIOQUIA', 'PUERTO NARI¥O', 'SAN CARLOS GUAROA', 'CUCUTA'}


  b) Aplicar coincidencia difusa para combinacion departamento + municipio en los dos dataframe

Realizar coincidencia difusa (fuzzy matching), significa que puede comparar dos cadenas de texto (strings) y medir su similitud, incluso si no son exactamente iguales. Esta técnica es muy útil en este caso porque si bien los nombres de los departamentos estan debidamente ajustados en los dos df,  los nombres de los municipios pueden tener diferencias (errores de tipeo, variantes en nombres, etc.).

El utilizar la columna del nombre del departamento, es importante en este caso porque en colombia existen municipios con el mismo nombre.


 - Instalar la thefuzz en el entorno de google colab, para realizar coincidencia difusa

In [38]:
!pip install thefuzz

- Aplicar thefuzz para coincidencia difusa

In [39]:
# Crear una clave única de departamento + municipio en ambos datasets
health_insurance_affiliated['dept_mpio'] = health_insurance_affiliated['dpr_nombre'] + '_' + health_insurance_affiliated['mnc_nombre']
dept_mpios_codes['dept_mpio'] = dept_mpios_codes['dept_name'] + '_' + dept_mpios_codes['mupio_name']

In [40]:
# Crear un diccionario de municipios y códigos basado en dept_mpios_codes
municipios_dict = dict(zip(dept_mpios_codes['dept_mpio'], dept_mpios_codes['dept_mpio_code']))

In [41]:
# Mapeo de cod_dane a los que tienen coincidencia exacta
health_insurance_affiliated['codigo_dane'] = health_insurance_affiliated['dept_mpio'].map(municipios_dict)

# Identificar los que no tienen coincidencia exacta (NaN)
no_match = health_insurance_affiliated[health_insurance_affiliated['codigo_dane'].isna()]

In [42]:
from thefuzz import process

# Func para aplicar fuzzy matching solo a los que no tienen coincidencia exacta
def get_best_match(row, municipios_dict, threshold=80):
    dept_mpio_concat = row['dpr_nombre'] + '_' + row['mnc_nombre']

    # Buscar la mejor coincidencia en dept_mpios_codes usando fuzzy matching
    best_match, score = process.extractOne(dept_mpio_concat, municipios_dict.keys())

    # Si la similitud supera el umbral definido, devolver el código mapeado
    if score >= threshold:
        return municipios_dict[best_match]
    else:
        return 'SIN COINCIDENCIA'

In [43]:
# Aplicar fuzzy matching a los que no tienen coincidencia exacta
health_insurance_affiliated.loc[health_insurance_affiliated['codigo_dane'].isna(), 'codigo_dane'] = no_match.apply(
    lambda row: get_best_match(row, municipios_dict, threshold=80),
    axis=1
)

  - Verificar resultados obtenidos

In [44]:
health_insurance_affiliated['codigo_dane'].apply(len).unique()

array([ 5, 16])

In [45]:
health_insurance_affiliated['codigo_dane'].apply(len).value_counts()

,count
codigo_dane,
5,1263971
16,124


Quedaron 124 registros que no encontraron coincidencia.

In [46]:
# Verificando registros que no coindirieros
print(len(health_insurance_affiliated[health_insurance_affiliated['codigo_dane']== 'SIN COINCIDENCIA']))

no_match = health_insurance_affiliated[health_insurance_affiliated['codigo_dane']== 'SIN COINCIDENCIA']

# Verificar a que departamentos pertenecen
no_match['dpr_nombre'].value_counts()

124


,count
dpr_nombre,
GUAINIA,98
AMAZONAS,26


In [47]:
# Ver una muestra de los primeros 5 registros de cada departamento
# Agrupar por 'dpr_nombre'
grouped = no_match.groupby('dpr_nombre')

# Iterar sobre cada grupo y mostrar los primeros 5 registros
for group_name, group_data in grouped:
    print(f"Grupo: {group_name}")
    print(group_data.head(5))
    print("\n")

Grupo: AMAZONAS
       tps_gnr_nombre tps_grp_etr_id  ent_id  \
288589       FEMENINO          NIÑEZ  EPSIC5   
413076       FEMENINO        ADULTEZ  EPSS41   
428891      MASCULINO          NIÑEZ  EPSS41   
468898      MASCULINO       JUVENTUD  EPSS41   
518128       FEMENINO          NIÑEZ  EPSS41   

                                          ent_nombre tps_rgm_nombre  \
288589  ENTIDAD PROMOTORA DE SALUD MALLAMAS EPSI -CM   CONTRIBUTIVO   
413076                                NUEVA EPS S.A.     SUBSIDIADO   
428891                                NUEVA EPS S.A.     SUBSIDIADO   
468898                                NUEVA EPS S.A.     SUBSIDIADO   
518128                                NUEVA EPS S.A.     SUBSIDIADO   

           tps_afl_nombre tps_est_afl_nombre tps_cnd_bnf_nombre zns_nombre  \
288589       BENEFICIARIO             ACTIVO          NO APLICA      RURAL   
413076  CABEZA DE FAMILIA             ACTIVO          NO APLICA      RURAL   
428891  CABEZA DE FAMILIA         

Procedemos a eliminar estos registros pues no existen pistas para saber de que municipio se trata.


In [48]:
# Eliminar registros SIN COINCIDENCIA EN cod Dane
health_insurance_affiliated = health_insurance_affiliated[health_insurance_affiliated['codigo_dane'] != 'SIN COINCIDENCIA']

In [49]:
# Verificar que solo quedan registros con cod_dane de 5 digitos
health_insurance_affiliated['codigo_dane'].apply(len).unique()

array([5])

In [50]:
health_insurance_affiliated['codigo_dane'].apply(len).value_counts()

,count
codigo_dane,
5,1263971


In [51]:
# Comparar listas de códigos
compare_lists(health_insurance_affiliated['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en health_insurance_affiliated", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en health_insurance_affiliated que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [52]:
health_insurance_affiliated.columns

Index(['tps_gnr_nombre', 'tps_grp_etr_id', 'ent_id', 'ent_nombre',
       'tps_rgm_nombre', 'tps_afl_nombre', 'tps_est_afl_nombre',
       'tps_cnd_bnf_nombre', 'zns_nombre', 'dpr_nombre', 'mnc_nombre',
       'tps_nvl_ssb_id', 'tps_grp_pbl_id', 'cantidad', 'dept_mpio',
       'codigo_dane'],
      dtype='object')

In [53]:
# Eliminar columnas que ya no son necesarias
deleted_col = ['dpr_nombre', 'mnc_nombre', 'dept_mpio']
health_insurance_affiliated = health_insurance_affiliated.drop(deleted_col, axis=1)

In [54]:
# Adicionar columna para trazabilidad de la fuente
# Crear una nueva columna 'source_id' basada en las condiciones
def asignar_source_id(row):
    if row['tps_rgm_nombre'] == 'CONTRIBUTIVO':
        return 112
    elif row['tps_rgm_nombre'] == 'SUBSIDIADO':
        return 113
    else:
        return None  # O cualquier otro valor por defecto que desees

health_insurance_affiliated['source_id'] = health_insurance_affiliated.apply(asignar_source_id, axis=1)

In [55]:
#Verificar asignacion de source_id
health_insurance_affiliated['source_id'].unique()

array([112, 113])

In [56]:
#Verificar asignacion de source_id
# Agrupar por 'tps_rgm_nombre'
grouped = health_insurance_affiliated.groupby('tps_rgm_nombre')

# Obtener el primer elemento de cada grupo
grouped.first()

,tps_gnr_nombre,tps_grp_etr_id,ent_id,ent_nombre,tps_afl_nombre,tps_est_afl_nombre,tps_cnd_bnf_nombre,zns_nombre,tps_nvl_ssb_id,tps_grp_pbl_id,cantidad,codigo_dane,source_id
tps_rgm_nombre,,,,,,,,,,,,,
CONTRIBUTIVO,MASCULINO,JUVENTUD,EPS048,ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SAL...,COTIZANTE,ACTIVO,NO APLICA,URBANA,NAN,NAN,27,23807,112
SUBSIDIADO,FEMENINO,VEJEZ,EPSS02,SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REG...,CABEZA DE FAMILIA,ACTIVO,NO APLICA,URBANA,N,NINOS-NINAS-ADOLESCENTES Y JOVENES EN PROCESO ...,3,50001,113


In [57]:
# Estructura final del df
health_insurance_affiliated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1263971 entries, 0 to 1264094
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   tps_gnr_nombre      1263971 non-null  object
 1   tps_grp_etr_id      1263971 non-null  object
 2   ent_id              1263971 non-null  object
 3   ent_nombre          1263971 non-null  object
 4   tps_rgm_nombre      1263971 non-null  object
 5   tps_afl_nombre      1263971 non-null  object
 6   tps_est_afl_nombre  1263971 non-null  object
 7   tps_cnd_bnf_nombre  1263971 non-null  object
 8   zns_nombre          1263971 non-null  object
 9   tps_nvl_ssb_id      1263971 non-null  object
 10  tps_grp_pbl_id      1263971 non-null  object
 11  cantidad            1263971 non-null  int64 
 12  codigo_dane         1263971 non-null  object
 13  source_id           1263971 non-null  int64 
dtypes: int64(2), object(12)
memory usage: 144.7+ MB


In [58]:
health_insurance_affiliated.shape

(1263971, 14)

## Salvar en archivo csv en el drive

In [59]:
# Guardar en archivos CSV en el drive
health_insurance_affiliated.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/health_insurance_affiliated_Adres.csv', index=False)